In [1]:
import re
import pickle
import sqlite3 as sl
from unicodedata import normalize
import pandas as pd
from pathlib import Path
import nltk
from nltk.corpus import stopwords
from sklearn.naive_bayes import BernoulliNB
from scipy import sparse
import numpy as np
nltk.download('stopwords')
con = sl.connect('../coleta_de_dados/banco_scraping_olx.db')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ANACLARASAMARINO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def normaliza_texto_vetorizado(text:str)->str: 
    frase_minuscula = normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII').lower()
    somente_palavras_validas = re.sub("([^\w]+)|((\w|d)*[\d](\w|d)*)"," ",frase_minuscula)
    texto_normalizado = re.sub("\s+"," ",somente_palavras_validas).strip()
    return texto_normalizado.split(" ")

In [3]:
todos_titulos = pd.read_sql("""
SELECT DISTINCT 
	url_anuncio,	
	titulo_anuncio	
FROM
	anuncios_resumo
""", con, chunksize=1000000)

In [4]:
def constroi_dicionario_palavras(df_pandas_chunked,dicionario):
    qt_linhas_processadas = 0
    for df in df_pandas_chunked:
        qt_linhas_processadas += len(df)
        print(f"qt linhas processadas: {qt_linhas_processadas}")
        for titulo in df['titulo_anuncio']:
            titulo_vetorizado = normaliza_texto_vetorizado(titulo)
            for palavras_titulo in titulo_vetorizado:
                if palavras_titulo:
                    if dicionario.get(palavras_titulo):
                        dicionario[palavras_titulo]+=1
                    else:
                        dicionario[palavras_titulo]=1

In [5]:
def filtrar_dicionario(dicionario_palavras,qtd_minima_palavras_usadas = 5):
    artigos_preposicoes = stopwords.words('portuguese')
    todas_chaves = dicionario_palavras.keys()
    chaves_para_excluir=[]
    for chave in todas_chaves:
        if chave in artigos_preposicoes or dicionario_palavras[chave] < qtd_minima_palavras_usadas or len(chave)<3:
            chaves_para_excluir.append(chave)
    for chave_ex in chaves_para_excluir:
        del dicionario_palavras[chave_ex]

In [6]:
dicionario_todas_palavras = {}

if not Path('./dicionario_todas_palavras.pkl').is_file():
    constroi_dicionario_palavras(todos_titulos,dicionario_todas_palavras)
    filtrar_dicionario(dicionario_todas_palavras,30)
    with open('./dicionario_todas_palavras.pkl', 'wb') as file:
        pickle.dump(dicionario_todas_palavras, file) 
else :
    with open('./dicionario_todas_palavras.pkl', 'rb') as file:
        dicionario_todas_palavras = pickle.load(file)

lista_todas_palavras = [*dicionario_todas_palavras.keys()]      

In [7]:
df_todas_categorias = pd.read_sql("""
SELECT DISTINCT 
	categoria_atual
FROM
	anuncios_resumo
""", con)
lista_todas_categorias = [*df_todas_categorias['categoria_atual']]

In [8]:
def categoria_para_indice(categoria,lista_todas_categorias):
    if lista_todas_categorias.count(categoria):
        return lista_todas_categorias.index(categoria)
    else:
        return -1
def titulo_para_indices(titulo,lista_todas_palavras):
    lista_palavras_titulo = normaliza_texto_vetorizado(titulo)
    indices_titulo =[]
    for palavra_titulo in lista_palavras_titulo:
        if lista_todas_palavras.count(palavra_titulo):
            indices_titulo.append(lista_todas_palavras.index(palavra_titulo))
    return indices_titulo                 

In [21]:
categoria_para_indice('Artigos infantis',lista_todas_categorias)
titulo_para_indices('baba de cachorro com casa grande',lista_todas_palavras)

[2, 1074, 323, 13]

In [10]:
lista_precos = [*range(10)] # coluna que ira classificar o preço (sera usado log 10)

In [11]:
def preco_para_indice(preco,indice_start=0):
    if isinstance(preco,float) or isinstance(preco,int):
        if preco>1:
            index = np.ceil(np.log10(preco))
            index = int(index) if index>=0 and index<len(lista_precos) else 0
            return index+indice_start
    return 0

In [12]:
def monta_sql_por_categoria(categoria):
    return f"""SELECT DISTINCT 
                    url_anuncio,
                    titulo_anuncio,
                    categoria_atual,
                    preco_anuncio
                FROM
                    anuncios_resumo
                WHERE categoria_atual = '{categoria}'
                LIMIT 1000 -- TIRAR isso DEPOIS
                """

In [13]:
classes_y = [*range(len(lista_todas_categorias))]

In [14]:
clf = BernoulliNB() # Modelo Usado (teste com Bernoulli)

In [15]:
for categoria in lista_todas_categorias:
    df_cat_atual = pd.read_sql(monta_sql_por_categoria(categoria),con)
    print(f'Treinando Modelo para a Categoria: {categoria}')
    qt_colunas = len(lista_todas_palavras)+len(lista_precos)
    X_linhas = []
    X_colunas = []
    for linha in range(len(df_cat_atual)): # Adiciona valores  na matrix espaça
        X_colunas_atual = titulo_para_indices(df_cat_atual['titulo_anuncio'][linha],lista_todas_palavras)
        X_colunas_atual += [preco_para_indice(df_cat_atual['preco_anuncio'][linha],len(lista_todas_palavras))]
        X_colunas+=X_colunas_atual
        X_linhas += [linha]*len(X_colunas_atual)
    Dados_X = np.full(len(X_linhas),1,dtype=np.int8)
    X = sparse.coo_matrix((Dados_X, (X_linhas, X_colunas)), shape=(len(df_cat_atual), qt_colunas))         
    Y = np.full(len(df_cat_atual),categoria_para_indice(df_cat_atual['categoria_atual'][0],lista_todas_categorias))
    clf.partial_fit(X,Y,classes=classes_y) #Treina o Modelo      

Treinando Modelo para a Categoria: Aluguel - casas e apartamentos
teste2.1
teste3
teste4
Treinando Modelo para a Categoria: Animais para agropecuária
teste2.1
teste3
teste4
Treinando Modelo para a Categoria: Antiguidades
teste2.1
teste3
teste4
Treinando Modelo para a Categoria: Aquários e acessórios
teste2.1
teste3
teste4
Treinando Modelo para a Categoria: Artigos infantis
teste2.1
teste3
teste4
Treinando Modelo para a Categoria: Barcos e aeronaves
teste2.1
teste3
teste4
Treinando Modelo para a Categoria: Beleza e saúde
teste2.1
teste3
teste4
Treinando Modelo para a Categoria: Bijouterias, relógios e acessórios
teste2.1
teste3
teste4
Treinando Modelo para a Categoria: Bolsas, malas e mochilas
teste2.1
teste3
teste4
Treinando Modelo para a Categoria: CDs, DVDs etc
teste2.1
teste3
teste4
Treinando Modelo para a Categoria: Cachorros
teste2.1
teste3
teste4
Treinando Modelo para a Categoria: Caminhões
teste2.1
teste3
teste4
Treinando Modelo para a Categoria: Carros, vans e utilitários
teste

In [16]:
# for categoria in lista_todas_categorias:
#     df_cat_atual = pd.read_sql(monta_sql_por_categoria(categoria),con)
#     print(f'Treinando Modelo para a Categoria: {categoria}')
#     qt_colunas = len(lista_todas_palavras)+len(lista_precos)
#     X = sparse.dok_matrix((len(df_cat_atual), qt_colunas), dtype=np.int8)

#     for linha in range(len(df_cat_atual)): # Adiciona valores  na matrix espaça
#         indices_palavras_titulo = titulo_para_indices(df_cat_atual['titulo_anuncio'][linha],lista_todas_palavras)
#         tupla_preco = (linha,preco_para_indice(df_cat_atual['preco_anuncio'][linha],len(lista_todas_palavras)))
#         X[linha,preco_para_indice(df_cat_atual['preco_anuncio'][linha],len(lista_todas_palavras))] = 1 # ref a faixa de precos
#         for indice_palavra in indices_palavras_titulo:
#             X[linha,indice_palavra] = 1
        
#     Y = np.full(len(df_cat_atual),categoria_para_indice(df_cat_atual['categoria_atual'][0],lista_todas_categorias))
#     clf.partial_fit(X,Y,classes=classes_y) #Treina o Modelo      

In [30]:
def procura_categoria(titulo,preco=None,qt_impr=5):
    qt_colunas = len(lista_todas_palavras)+len(lista_precos)
    XN = sparse.dok_matrix((1, qt_colunas), dtype=np.int8)
    if preco:
        XN[0,preco_para_indice(preco,len(lista_todas_palavras))] = 1
    indices_palavras_titulo = titulo_para_indices(titulo,lista_todas_palavras)
    for indice_palavra in indices_palavras_titulo:
        XN[0,indice_palavra] = 1
    categorias_prob = clf.predict_proba(XN[0])
    indice_categorias_ordenado = np.flip(np.argsort(categorias_prob))
    limite_impressoes = qt_impr
    for indice_categoria in indice_categorias_ordenado.tolist()[0]:
        # print(indice_categoria)
        print(f'{lista_todas_categorias[indice_categoria]}:{categorias_prob[0][indice_categoria]*100:.2f}%')
        if limite_impressoes<=0:
            break
        limite_impressoes -=1

In [39]:
procura_categoria('celta novo',20000)

Carros, vans e utilitários:90.10%
Peças para carros, vans e utilitários:1.96%
Caminhões:1.05%
Peças para tratores e máquinas:1.02%
Máquinas para produção industrial:0.95%
Ônibus:0.89%
